In [1]:
import ibis
from rbc.mapd import RemoteMapD

In [2]:
mapd = RemoteMapD()  # RemoteMapD(user=, password=, host=, port=, dbname=)
mapd_ibis = ibis.mapd.connect(user=mapd.user, password=mapd.password,
                              host=mapd.host, port=mapd.port, database=mapd.dbname)

In [3]:
# ibis.mapd bug workaround:
import sys
sys.modules['ibis.mapd.operations']._sql_type_names['float32'] = 'float'

In [4]:
schema = ibis.schema([('x', 'float'), ('y', 'double'), ('i', 'int32'), ('j', 'int64')])
print(schema)

ibis.Schema {  
  x  float32
  y  float64
  i  int32
  j  int64
}


In [5]:
# Create test table
mapd_ibis.drop_table('mytable')
mapd_ibis.create_table('mytable', schema=schema)
mytable = mapd_ibis.table('mytable')
for i in range(5):
    mapd.sql_execute('insert into mytable values (%s, %s, %s, %s)' % (i, i+10, i+100, i+1000))
mytable.execute()

,x,y,i,j
0,0.0,10.0,100,1000
1,1.0,11.0,101,1001
2,2.0,12.0,102,1002
3,3.0,13.0,103,1003
4,4.0,14.0,104,1004


In [6]:
# Define and register MapD Runtime UDF
@mapd('int32(int32)')
def incr(x):
    return x + 1
@mapd('int64(int64)')
def incr(x):
    return x + 10
mapd.register()

In [7]:
t = mapd_ibis.sql('select i, incr(i), j, incr(j) from mytable')
expr = t[t.i < 103]
print(expr.compile())

SELECT *
FROM (
  select i, incr(i), j, incr(j) from mytable
) t0
WHERE "i" < 103


In [8]:
expr.execute()

,i,EXPR$1,j,EXPR$3
0,100,101,1000,1010
1,101,102,1001,1011
2,102,103,1002,1012
